In [1]:
#Result-GPT для Doom. Но теперь с памятью. И с дескрипшнами перед принятием решений
import sys
import os
import importlib
#root = './drive/My Drive/Colab Notebooks/rl/'
root = './'
sys.path.insert(1, root + 'env')
sys.path.insert(1, root + 'agents')
sys.path.insert(1, root + 'common')

import gym
import pylab
import random
import numpy as np
from collections import deque


import time
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd
import pickle
import copy
import vizdoom
from vizdoom import *
import cv2
from os.path import exists
import codecs

import torch
from transformers import GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM


E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
device = torch.device('cuda:0')
#device = torch.device('cpu', 0)

In [3]:
EPISODES=5
episode_len = 1900#1900
subtraj_len = 130
#subtraj_selection = 450#1000
subtraj_selection = 2
recoms_count = 0
#recoms_count = 0
actions_count = 11
frc_count = 10
frc_len = 40

subtraj_len_dense = 130#50
recoms_dense_period = 1
recoms_count_dense = 4000

proba_desc = 0.03

In [4]:
#<<image>> summarize: (обычное описание)
#(summary) <<image>> plan: (обычный план)
#А (summary) <<image>> summarize: history: (прошлое summary); (обычное описание)
#ИЛИ
#Б (summary) <<image>> summarize: history: (delta summary); (обычное описание)
#А проще, а если summary ещё и обрезать, то А вполне реализуемо

In [5]:
#!pip install numpy==1.23.5

In [ ]:
#Я не знаю, где они лежат, надо искать
data_path = '../gpt/chatgpt/data/all_txt_cut.txt'
model_name = "../gpt/chatgpt/gpt2_finetuned"

In [6]:
import codecs
with codecs.open(data_path, 'r', 'utf8', errors='ignore') as f:
    lines = f.readlines()
lines = ''.join(lines)
lines[:1000]

'<IN><714><872><373><545><698><887><803><718><40><790><505><692><40><751><269><597><587><514><255><891><870><556><706><208><897><362><937><572><330><358><626><635><830><362><910><751><553><966><414><758><219><125><928><887><526><297><313><878><806><922><598><582><358><308><7><433><66><172><202><44><192><528><994><301><780><539><316><80><988><523><630><585><500><32><172><642><746><109><590><379><661> description:<OUT><OUT> serious sam, machinegun in hands, shooting, klir right-ahead, klir ahead, kilr at left, purple zombie far at right, blue ammo box far left-ahead<END>\n\n<IN><714><872><373><545><698><887><803><718><40><790><505><692><40><751><269><597><587><514><255><891><870><556><706><208><897><362><937><572><330><358><626><635><830><362><910><751><553><966><414><758><219><125><928><887><526><297><313><878><806><922><598><582><358><308><7><433><66><172><202><44><192><528><994><301><780><539><316><80><988><523><630><585><500><32><172><642><746><109><590><379><661> kill all plan:<OUT>

In [7]:
# 1. load a pretrained model

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.to(device)

Some weights of the model checkpoint at ../gpt/chatgpt/gpt2_finetuned were not used when initializing GPT2LMHeadModel: ['v_head.2.bias', 'v_head.4.weight', 'v_head.4.bias', 'v_head.0.weight', 'v_head.0.bias', 'v_head.2.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51785, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51785, bias=False)
)

In [8]:
reward2substr = {}
step = 0.1
max_val = 25
min_val = -25
for val in np.arange(min_val, max_val, step):
    val = np.round(val, 2)
    key = f'<r{val}>'
    reward2substr[val] = key
print(list(reward2substr.keys())[0],'-',list(reward2substr.keys())[-1])

-25.0 - 24.9


In [9]:
def reward_make_substr(r):
    if r in list(reward2substr.keys()):
        return reward2substr[r]
    else:
        if r>max_val:
            r = max_val - 0.1
        elif r<min_val:
            r = min_val
        r = np.round(r, 1)
        return reward2substr[r]

In [10]:
#Учим эту модель. У Nehc есть пример, как это делать
#Модификация токенайзера 
#добавляем несколько токенов нашей разметки

#сюда надо фигануть весь словарь картинок
video_tokens_cnt = 1024
video_tokens = []
for i in range(video_tokens_cnt):
    video_tokens.append(f'<{i}>')
    
special_tokens_dict = {'additional_special_tokens': video_tokens + ['<IN>','<OUT>','<END>','|PAD|'] + list(reward2substr.values())}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
print(tokenizer.all_special_tokens)
tokenizer.pad_token = '|PAD|'

['<|endoftext|>', '|PAD|', '<0>', '<1>', '<2>', '<3>', '<4>', '<5>', '<6>', '<7>', '<8>', '<9>', '<10>', '<11>', '<12>', '<13>', '<14>', '<15>', '<16>', '<17>', '<18>', '<19>', '<20>', '<21>', '<22>', '<23>', '<24>', '<25>', '<26>', '<27>', '<28>', '<29>', '<30>', '<31>', '<32>', '<33>', '<34>', '<35>', '<36>', '<37>', '<38>', '<39>', '<40>', '<41>', '<42>', '<43>', '<44>', '<45>', '<46>', '<47>', '<48>', '<49>', '<50>', '<51>', '<52>', '<53>', '<54>', '<55>', '<56>', '<57>', '<58>', '<59>', '<60>', '<61>', '<62>', '<63>', '<64>', '<65>', '<66>', '<67>', '<68>', '<69>', '<70>', '<71>', '<72>', '<73>', '<74>', '<75>', '<76>', '<77>', '<78>', '<79>', '<80>', '<81>', '<82>', '<83>', '<84>', '<85>', '<86>', '<87>', '<88>', '<89>', '<90>', '<91>', '<92>', '<93>', '<94>', '<95>', '<96>', '<97>', '<98>', '<99>', '<100>', '<101>', '<102>', '<103>', '<104>', '<105>', '<106>', '<107>', '<108>', '<109>', '<110>', '<111>', '<112>', '<113>', '<114>', '<115>', '<116>', '<117>', '<118>', '<119>', '<1

In [11]:
#специальные токены { vertical-output: true }
T_OUT = tokenizer.encode('<OUT>')[0]
T_END = tokenizer.encode('<END>')[0]
T_PAD = tokenizer.encode('|PAD|')[0]
T_124 = tokenizer.encode('<124>')[0]
#reward-токен:
rmin = list(reward2substr.values())[0]
rmax = list(reward2substr.values())[-1]
T_RMIN = tokenizer.encode(rmin)[0]
T_RMAX = tokenizer.encode(rmax)[0]
print(rmin, rmax)
T_OUT, T_END, T_PAD, T_124, T_RMIN, T_RMAX

<r-25.0> <r24.9>


(51282, 51283, 51284, 50381, 51685, 51784)

In [12]:
#VQ-GAN 1024 
Model = "f16_1024" #param ["f16_1024", "f16_16384", "f16_16384_hf"]
import cv2
import torch
import yaml
import torch
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from omegaconf import OmegaConf
from taming.models.vqgan import VQModel

def load_config(config_path, display=False):
  config = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config)))
  return config

def load_vqgan(config, ckpt_path=None):
  model = VQModel(**config.model.params)
  if ckpt_path is not None:
    sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
    missing, unexpected = model.load_state_dict(sd, strict=False)
  return model.eval()

def preprocess_vqgan(x, roll=True):
  x = 2.*x - 1.
  if roll:
    x = np.rollaxis(x,3,1)
  x = torch.Tensor(x)
  return x

def preprocess(x, permt=True):
  if permt:
    x = x.permute(0,2,3,1).numpy()
  x = np.clip(x, -1., 1.)
  x = (x + 1.)/2.
  return x

def custom_to_pil(x):
  x = np.clip(x, -1., 1.)
  x = (x + 1.)/2.
  x = (255*x).astype(np.uint8)
  x = Image.fromarray(x)
  if not x.mode == "RGB":
    x = x.convert("RGB")
  return x

vq_conf = load_config(f"../gpt/chatgpt/chk_points/vqgan_imagenet_{Model}.yaml", display=False)
vq_model = load_vqgan(vq_conf, ckpt_path=f"../gpt/chatgpt/chk_points/vqgan_imagenet_{Model}.ckpt").to(device)


Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips\vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


In [13]:
imsize = [16*9, 16*9]

In [14]:
num_to_action_desc = {2:'fire', 0:'strafe left', 5:'turn left', 1:'strafe right', 6:'turn right', 3:'move ahead', 7:'use', 4:'move back'}
action_to_num_desc = {num_to_action_desc[k] : k for k in num_to_action_desc}
action_to_num_desc

{'fire': 2,
 'strafe left': 0,
 'turn left': 5,
 'strafe right': 1,
 'turn right': 6,
 'move ahead': 3,
 'use': 7,
 'move back': 4}

In [15]:
def answer(text, max_tokens=5, random=False):
    context_max_size = 256
    #text = '\n' + text
    inpt = tokenizer.encode(text, return_tensors="pt")
    inpt = tokenizer.encode(text, return_tensors="pt")

    inpt = inpt[-context_max_size:]
    inpt = inpt.to(device)

    gener_len = max_tokens
    tokens_use = 5
    #out = model.generate(inpt,  max_length=len(inpt[0])+300, do_sample=True, top_k=5, top_p=0.95, temperature=1, eos_token_id=T_END, pad_token_id=T_PAD)
    temperature = 0.01
    temperature = 0.9
    if random and not (np.random.rand()<0.1):
        random = False

    if random:
        out = model.generate(inpt,  max_length=len(inpt[0])+gener_len, do_sample=True, top_k=1, top_p=0.95, temperature=0.1, eos_token_id=T_END, pad_token_id=T_PAD)
    else:
        out = model.generate(inpt,  max_length=len(inpt[0])+gener_len, do_sample=False, top_k=1, top_p=0.95, temperature=0.01, eos_token_id=T_END, pad_token_id=T_PAD)
        
    out_tokens = torch.where(out[0]==T_OUT)
    #last_repl = out[0][out_tokens[0][-1]+1:-1]
    #print(out)
    out_tokens = out_tokens[-tokens_use:][:max_tokens]
    last_repl = out[0][out_tokens[0][-1]+1:-1]
    repl = tokenizer.decode(last_repl)

    return repl
            
    

In [16]:
#!sudo apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
#nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
#libopenal-dev timidity libwildmidi-dev unzip
#!sudo apt-get install libboost-all-dev
#!pip install vizdoom

In [17]:
#как сделать запись в файл? Перегони в картинку, её перегони в верный формат.

In [18]:
#(224, 224, 3) - размеры resnet101, resnet 101v2

In [19]:
# Creates and initializes ViZDoom environment.
def initialize_vizdoom(config_file_path,doom_map="map01"):
    print("Initializing doom...")
    game = DoomGame()
    game.load_config(config_file_path)
    game.set_doom_map(doom_map)
    game.set_screen_format(ScreenFormat.RGB24)
    game.set_screen_resolution(ScreenResolution.RES_200X150)
    game.set_depth_buffer_enabled(True)
    # Enables depth buffer.
    # Enables labeling of in game objects labeling.
    game.set_labels_buffer_enabled(True)
    # Enables buffer with top down map of the current episode/level.
    game.set_automap_buffer_enabled(True)
    game.set_depth_buffer_enabled(True)
    # Enables information about all objects present in the current episode/level.
    #game.set_objects_info_enabled(True)
    # Enables information about all sectors (map layout).
    #game.set_sectors_info_enabled(True)
    # Sets other rendering options (all of these options except crosshair are enabled (set to True) by default)
    game.set_render_hud(True)
    #game.set_render_hud(False)
    #game.set_render_minimal_hud(False)  # If hud is enabled
    #game.set_render_minimal_hud(True)  # If hud is enabled
    game.set_render_crosshair(False)
    game.set_render_weapon(True)
    #game.set_render_weapon(False)
    
    game.set_render_decals(False)  # Bullet holes and blood on the walls
    game.set_render_particles(False)
    game.set_render_effects_sprites(True)  # Smoke and blood
    game.set_render_messages(False)  # In-game messages
    game.set_render_corpses(True)
    game.set_render_screen_flashes(True)  # Effect upon taking damage or picking up items
    #game.set_objects_info_enabled(True)
    # Adds buttons that will be allowed.
    lst_acts = [Button.ATTACK,Button.USE,Button.MOVE_LEFT,Button.MOVE_RIGHT,Button.MOVE_FORWARD,Button.MOVE_BACKWARD,Button.TURN_LEFT,Button.TURN_RIGHT]#,Button.SELECT_NEXT_WEAPON]
    for action in lst_acts:
        game.add_available_button(action)
    # Adds game variables that will be included in state.
    game.add_available_game_variable(GameVariable.SELECTED_WEAPON)
    game.add_available_game_variable(GameVariable.AMMO2)
    game.add_available_game_variable(GameVariable.HEALTH)
    game.add_available_game_variable(GameVariable.FRAGCOUNT)
    game.add_available_game_variable(GameVariable.KILLCOUNT)
    game.add_available_game_variable(GameVariable.ITEMCOUNT)
    game.add_available_game_variable(GameVariable.HITCOUNT)
    # Causes episodes to finish after 200 tics (actions)
    game.set_episode_timeout(episode_len)
    # Makes episodes start after 10 tics (~after raising the weapon)
    game.set_episode_start_time(10)
    # Makes the window appear (turned on by default)
    game.set_window_visible(True)
    # Turns on the sound. (turned off by default)
    game.set_sound_enabled(False)
    # Sets the living reward (for each move) to -1
    game.set_living_reward(1)
    # Sets ViZDoom mode (PLAYER, ASYNC_PLAYER, SPECTATOR, ASYNC_SPECTATOR, PLAYER mode is default)
    #game.set_mode(Mode.PLAYER)
    game.set_mode(Mode.PLAYER)
    # Enables engine output to console.
    #game.set_console_enabled(True)
    # Initialize the game. Further configuration won't take any effect from now on.
    time.sleep(1)
    game.init()
    print("Doom initialized.")
    return game

In [20]:
black_square = np.zeros((224, 224, 3))

In [21]:
doom=1

print('_____',pd.Timestamp.now())

env = initialize_vizdoom(root + 'doom_files/scenarios/doom2.cfg',doom_map="e1m1")
#env = initialize_vizdoom(root + 'doom_files/scenarios/basic.cfg',doom_map="map06")
#[Button.ATTACK,Button.USE,Button.MOVE_LEFT,Button.MOVE_RIGHT,Button.MOVE_FORWARD,Button.MOVE_BACKWARD,Button.TURN_LEFT,Button.TURN_RIGHT]
actions = [[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]]


state_size = (env.get_screen_height(),env.get_screen_width(),3)

action_size = len(actions)


_____ 2023-10-03 20:23:40.642587
Initializing doom...
Doom initialized.


In [22]:
#np.shape(state) # (150, 200, 3)world model mse

In [23]:
def prepare_features(state, verbose=False,extended=False):
    if state is None:
        return np.zeros([1,15])
    shape = np.shape(state.labels_buffer)
    shape = [shape[1],shape[0]]
    enemy_types = ['PainElemental','FloatingSkull','Mastermind','MasterMind','Cyberdemon','CyberDemon','BaronOfHell','HellKnight','LostSoul','Archvile','Revenant','Arachnotron','Cacodemon','Fatso','ChaingunGuy','ShotgunGuy','DoomImp','DoomPinky','Zombieman','Spectre','Demon']
    item_types = ['BFGExtra','RedSkull','BlueSkull','YellowSkull','Allmap','RadSuit','Chainsaw','InvulnerabilitySphere','BFG9000','RocketBox','ClipBox','YellowCard','RedCard','CellPack','Cell','Backpack','ShellBox','RocketLauncher','RocketAmmo','PlasmaRifle','Megasphere','Chaingun','BlurSphere','Berserk','HealthBonus','Stimpack','Medikit','Shotgun','Chaingun','ArmorBonus','BlueCard','Clip','BlueArmor','GreenArmor','Shell','Soulsphere','SuperShotgun']
    weapon_types = ['Chainsaw','BFG9000','RocketLauncher','PlasmaRifle','Chaingun','Shotgun','Chaingun','BlueArmor','InvulnerabilitySphere','Soulsphere','SuperShotgun']
    projectile_types = ['ArchvileFire','RevenantTracer','ArachnotronPlasma','DoomImpBall','FatShot','CacodemonBall','BaronBall']
    #проверить наличие объектов обоих типов 
    d = {'enemy_fwd':0,'enemy_lft':0,'enemy_rgt':0,'enemy_vlft':0,'enemy_vrgt':0,
        'item_fwd':0,'item_lft':0,'item_rgt':0,'item_vlft':0,'item_vrgt':0,
        'projectile_fwd':0,'projectile_lft':0,'projectile_rgt':0,'projectile_vlft':0,'projectile_vrgt':0,
        'weapon_fwd':0,'weapon_lft':0,'weapon_rgt':0,'weapon_vlft':0,'weapon_vrgt':0
        }
    obj_list = []
    obj_matrix = np.zeros([shape[1],shape[0],6])
    obj_matrix[:,:,-1] = state.depth_buffer
    description = ''
    description_enemies = ''
    description_items = ''
    description_projectiles = ''
    description_corpses = ''
    for obj in state_raw.labels:
        x = int(obj.x)
        y = int(obj.y)
        w = int(obj.width)
        h = int(obj.height)
        if np.abs(obj.x-shape[0]*0.5)<6:
            where = 'ahead'
        elif ((obj.x-shape[0]*0.5)>=6) and ((obj.x-shape[0]*0.5)<40):
            where = 'right-ahead'
        elif (obj.x-shape[0]*0.5)>=40:
            where = 'at right'
        elif ((obj.x-shape[0]*0.5)<=-6) and ((obj.x-shape[0]*0.5)>-40):
            where = 'left-ahead'
        elif (obj.x-shape[0]*0.5)<=-40:
            where = 'at left'
            
        obj_alias = obj.object_name
        obj_alias = obj_alias.replace('Zombieman', 'rifleman zombie').replace('ShotgunGuy', 'shotgun zombie')
        obj_alias = obj_alias.replace('SuperShotgun', 'double-barrel').replace('DoomImp', 'imp').replace('Fatso','mancubus')
        obj_alias = obj_alias.replace('FloatingSkull', 'lostsoul').replace('DoomPinky', 'pinky').replace('impBall','fireball')
        obj_alias = obj_alias.replace('RevenantTracer', 'rocket').replace('DoomImpBall', 'fireball').replace('CacodemonBall','fireball')
        obj_alias = obj_alias.replace('BaronBall', 'green projectile').replace('BaronOfHell', 'hell baron').replace('YellowCard','yellow key')
        obj_alias = obj_alias.replace('YellowCard','yellow key').replace('BlueCard','blue key').replace('RedCard','red key').replace('FatShot','fireball').replace('ArachnotronPlasma','green plasma').replace('PlasmaRifle','plasmagun')
        obj_alias = obj_alias.replace('Dead','dead ')
        
        if obj.object_name in enemy_types:
            obj_type = 'enemy'
            obj_type_num = 0
            description_enemies += f'{obj_alias} {where},'
        elif obj.object_name in weapon_types:
            obj_type = 'weapon'
            obj_type_num = 5
            description_items += f'{obj_alias} {where},'
        elif obj.object_name in item_types:
            obj_type = 'item'
            obj_type_num = 1
            description_items += f'{obj_alias} {where},'
        elif obj.object_name in projectile_types:
            obj_type = 'projectile'
            obj_type_num = 2
            description_projectiles += f'{obj_alias} {where},'
        elif 'Dead' in obj.object_name:
            obj_type = 'corpse'
            obj_type_num = 3
            description_corpses += f'{obj_alias} {where},'
        else:
            if obj.object_name!='DoomPlayer':
                obj_type = ''
                obj_type_num = 4
            else:
                continue
        
        wp_name = GameVariable.SELECTED_WEAPON.value
        if wp_name==15:
            wp_name='pistol'
        elif wp_name==16:
            wp_name='shotgun'
        elif wp_name==17:
            wp_name='double-barrel'
        elif wp_name==14:
            wp_name='nothing'
        elif wp_name==18:
            wp_name='chaingun'
        
        
        #f'{wp_name} in hands,' +
        description = description_enemies + description_projectiles + description_items + description_corpses
        description = description[:-1]
        
        
        rec = {'obj_type':obj_type,'obj_name':obj.object_name,'x':obj.x,'y':obj.y}
        obj_list.append(rec)
        obj_matrix[y:y+h,x:x+w,obj_type_num] = 1
        if obj_type!='':
            if np.abs(obj.x-shape[0]*0.5)<6:
                d[obj_type+'_fwd']=1
            elif ((obj.x-shape[0]*0.5)>=6) and ((obj.x-shape[0]*0.5)<40):
                d[obj_type+'_rgt']=1
            elif (obj.x-shape[0]*0.5)>=40:
                d[obj_type+'_vrgt']=1
            elif ((obj.x-shape[0]*0.5)<=-6) and ((obj.x-shape[0]*0.5)>-40):
                d[obj_type+'_lft']=1
            elif (obj.x-shape[0]*0.5)<=-40:
                d[obj_type+'_vlft']=1            
    #if np.random.rand()<0.1:
    #    print(description)
    if verbose:
        print(d)
    if extended:
        return np.array(list(d.values()),ndmin=2), obj_list, obj_matrix, description
    else:
        return np.array(list(d.values()),ndmin=2)

In [24]:
ACTION_PERIOD = 8

In [25]:
class ResultGPT:
    def __init__(self, mode='one_frame_mode'):
        self.max_summary_write = 30
        self.max_summary_show_simbols = 900#900*2
        self.plan_max_tokens = 9
        self.task = 'kill all'
        self.prev_summary = ''
        #если да, то мы генерим description на базе summary и картинки (под это надо специально учить нейронку)
        self.use_complex_summary = False
        #если да, то мы генерим план на базе summary и картинки (под это надо специально учить нейронку)
        self.plan_complex_summary = False#False
        #если да, то просим summary, иначе description
        self.ask_summary = False
        
        #добавить дескрипшн после токенов картинки
        self.descriptions_to_history = True#в логах
        self.descriptions_to_actions = True#False#в фактических действиях
        #
        self.critical_trials = 1
        self.plan_max_check_tokens = 40
        
        #стоит ли искать точный аналог в базе примеров
        self.report_fullstring_search = True
        
        self.actions_from_keyboard = False
        
        
        #mode = 'one_frame_mode'# 'hand_mode' 'simple_test_mode' 'full_test_mode' 'one_frame_mode'
        
        self.deterministic_memory = False
        self.deterministic_plan = False#True
        
        if mode == 'hand_mode':
            #Принимаем решения вручную
            self.ask_summary = False
            self.use_complex_summary = False
            self.plan_complex_summary = False
            self.actions_from_keyboard = True
            self.critical_trials = 1
        if mode == 'simple_test_mode':
            #Принимаем решения с учетом памяти, но генерим память через description
            self.ask_summary = False
            self.use_complex_summary = False
            self.plan_complex_summary = True
            self.actions_from_keyboard = False
        if mode == 'full_test_mode':
            #Принимаем решения с учетом памяти, генерим память правильным образом
            self.ask_summary = True
            self.use_complex_summary = True
            self.plan_complex_summary = True
            self.actions_from_keyboard = False
        if mode == 'one_frame_mode':
            #Принимаем решения без учёта памяти
            self.ask_summary = False
            self.use_complex_summary = False
            self.plan_complex_summary = False
            self.actions_from_keyboard = False
            self.deterministic_memory = True
            self.deterministic_plan = False
        
        self.random_step_proba = 0.03
    def act(self, img):
        verbose = False
        action = int(np.round(np.random.rand()*7))    
        
        img_orig = np.array(cv2.resize(img, imsize), dtype=np.float32)/255.
        img = preprocess_vqgan(np.stack(1*[img_orig]), True)       

        z, _, [_, _, ind] = vq_model.encode(img.to(device))
        ind.squeeze_()
        b,c,h,w = z.shape
        token_string = '<' + '><'.join( [str(el) for el in list(ind.detach().cpu().numpy())] ) + '>'

        #запоминаем
        if self.ask_summary:
            description_name = 'summarize'
        else:
            description_name = 'description'
        text_for_summary_complex = f"<IN>{self.prev_summary}{token_string} {description_name}:<OUT>"
        text_for_summary_simple = f"<IN>{token_string} {description_name}:<OUT>"
        self.desc_simple = answer(text_for_summary_simple, self.max_summary_write, random=self.deterministic_memory)
        
        answer(text_for_summary_simple, self.max_summary_write, random=self.deterministic_memory)
       
        
        if self.use_complex_summary:
            self.summary = answer(text_for_summary_complex, self.max_summary_write, random=self.deterministic_memory)
            if verbose:
                print('asked cplx summary:  ',text_for_summary_complex)
                print('answer:  ', self.summary)
                index = lines.find(text_for_summary_complex)
                offset = 150
                if index>=0:
                    print('ans in DATASET:', lines[index + 81*4 - 1: index + 81*4 + offset])
                    
            self.summary_simple = ''
        else:
            #self.summary = answer(text_for_summary_simple, self.max_summary_write, random=self.deterministic_memory)
            self.summary = self.desc_simple
            if verbose:
                print('asked descr summary:  ',text_for_summary_simple)
                print('answer:  ', self.summary)
                index = lines.find(text_for_summary_simple)
                offset = 150
                if index>=0:
                    print('ans in DATASET:', lines[index + 81*4 - 1: index + 81*4 + offset])
            self.summary_simple = self.summary
            self.summary = self.prev_summary + ';' + self.summary
            #if verbose:
            #    print('final summary', self.summary)
            
        
        #планируем
        if self.descriptions_to_actions:
            self.desc_simple_if_need = self.desc_simple
        else:
            self.desc_simple_if_need = ''
        text_for_plan_complex = f"<IN>{self.prev_summary}{token_string}{self.desc_simple_if_need} {self.task} plan:<OUT>"
        text_for_plan_simple = f"<IN>{token_string}{self.desc_simple_if_need} {self.task} plan:<OUT>"
        
        if self.descriptions_to_history:
            self.desc_simple_if_need = self.desc_simple
        else:
            self.desc_simple_if_need = ''
        text_for_plan_complex4logs = f"<IN>{self.prev_summary}{token_string}{self.desc_simple_if_need} {self.task} plan:<OUT>"
        text_for_plan_simple4logs = f"<IN>{token_string}{self.desc_simple_if_need} {self.task} plan:<OUT>"
        if self.actions_from_keyboard:
            #num_to_action_desc = {2:'fire', 0:'strafe left', 5:'turn left', 1:'strafe right', 6:'turn right', 3:'move ahead', 7:'use', 4:'move back'}
            key_to_action = {'w':'move ahead', 'a':'turn left', 's':'move back', 'd':'turn right', 'q':'strafe left', 'e':'strafe right', 'f':'use', ' ':'fire'}
            key = input()
            if key in key_to_action.keys():
                action = key_to_action[key]
            else:
                action = num_to_action_desc[int(np.round(np.random.rand()*7))]
            plan = action + ','
        else:
            df_plans = []
            for i in range(self.critical_trials):
                if self.plan_complex_summary:
                    plan = answer(text_for_plan_complex, self.plan_max_tokens, random=self.deterministic_plan)
                    if verbose:
                        print('asked plan:  ',text_for_plan_complex)
                        print('ans plan:  ',plan)
                else:
                    plan = answer(text_for_plan_simple, self.plan_max_tokens, random=self.deterministic_plan)
                    if verbose:
                        print('asked plan:  ',text_for_plan_simple)
                        print('ans plan:  ',plan)
                if self.critical_trials > 1:
                    try:
                        #проверить
                        #f'<<{img_name1}>> plan:{plan} forecast vars: kills:{kills}, hits:{hits}, delta hp:{dhp}, items:{items}<END>\n'
                        check_ask = f'<IN>{token_string} plan:{plan} forecast vars:<OUT>'
                        estimation = answer(check_ask, self.plan_max_check_tokens)
                        estimation = estimation.replace(' ', '')
                        estimation_lst = estimation.split(',')
                        estimation_lst = estimation_lst[:6]
                        estimation_d = {el.split(':')[0]:el.split(':')[1] for el in estimation_lst}
                        if verbose:
                            print('estimation', estimation)
                        rec = {'kills':estimation_d['kills'], 'hits':estimation_d['hits'], 'plan':plan}
                        df_plans.append(rec)
                    except Exception:
                        pass
            if self.critical_trials > 1:
                try:
                    df_plans = pd.DataFrame(df_plans)
                    df_plans['kills'] = df_plans['kills'].astype(float)
                    df_plans['hits'] = df_plans['hits'].astype(float)
                    df_plans['q'] = df_plans['kills'] + df_plans['hits']*0.3
                    amax = df_plans.q.argmax()
                    plan = df_plans.plan.iloc[amax]
                    if verbose:
                        print('df_plans', df_plans)
                        print('plan', plan)
                    #if df_plans['q'].max() != df_plans['q'].min():
                    #    print('critical: made plan q=', df_plans['q'].max(), 'qmin=', df_plans['q'].min())
                except Exception:
                        pass
                    
            

        plan_action_idx = plan.find(',')
        if plan_action_idx>0:
            plan_action = plan[:plan_action_idx]
            if verbose:
                print('plan_action:  ',plan_action)

            for key in action_to_num_desc.keys():
                if key in plan_action:
                    #print(key, 'plan', plan)
                    action = action_to_num_desc[key]
                    if verbose:
                        print('key_action:  ',action)
                    break
            
            self.summary += '|' + plan_action + ' '
            if len(self.summary)>self.max_summary_show_simbols:
                self.summary = self.summary[-self.max_summary_show_simbols:]
            self.prev_summary = self.summary
        

        if self.report_fullstring_search:
            if self.plan_complex_summary:
                text = text_for_summary_complex
            else:
                text = text_for_summary_simple
            index = lines.find(text)
            #print('index', index)
            offset = 150
            if index>=0:
                print('plan in DATASET:', lines[index + 81*4 - 1: index + 81*4 + offset])
        
        gpt_memory_report = {'text_for_plan_complex': text_for_plan_complex4logs, 'plan': plan, 'text_for_summary_complex': text_for_summary_complex, 'text_for_summary_simple': text_for_summary_simple, 'summary': self.summary, 'summary_simple': self.summary_simple}
        
        if np.random.rand() < self.random_step_proba:
            action = int(np.round(np.random.rand()*7))
        if verbose:
            print('final key_action:  ',action)
        return action, gpt_memory_report

In [26]:
def gpt_strategy_inertial(img, action_prev=None, t_now=None):
    #почему-то 0 - стрейф влево, 1 - стрейф вправо, 2 - огонь, 3 - use, 4 - вперёд, 5 - turn_right, 6 - turn_left, 7 - back
    if t_now is None:
        t_now = 0
    if t_now%ACTION_PERIOD==0:
        action, gpt_memory_report = agent.act(img)
        #action = plan_picture(img, task='scouting')
        #print(num_to_action_desc[action])
        return action, gpt_memory_report
    else:
        return action_prev, None

In [27]:
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_3.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_5.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_7.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_8.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_9.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_10.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_11.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_12.txt'
#
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_13.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_14.txt'

path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_15.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_16.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_17.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_18.txt'

path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_19.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_20.txt'
path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_21.txt'

In [28]:
#path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_hum.txt'

In [ ]:
items_set = []
doom_type = 2
for mapnum in range(3,33):
#for mapnum in range(5,31):
    if doom_type==1:
        e_num = mapnum//10 + 1
        mapnum = mapnum%10
        if mapnum==0:
            mapnum=1
        mapname = f'e{e_num}m{mapnum}'
    else:
        mapname = str(mapnum)
        if mapnum<10:
            mapname = '0' + mapname
        mapname = 'map' + mapname
    #env = initialize_vizdoom(root + 'doom_files/scenarios/doom1.cfg',doom_map=mapname)
    env = initialize_vizdoom(root + 'doom_files/scenarios/doom2.cfg',doom_map=mapname)
    #env = initialize_vizdoom(root + 'doom_files/scenarios/plutonia.cfg',doom_map=mapname)
    
    logs_episodes = []
    for ep in range(EPISODES):
        rnd = np.random.rand()
        #'hand_mode' 'simple_test_mode' 'full_test_mode' 'one_frame_mode'
        if rnd<0.35:
            mode = 'simple_test_mode'
        elif rnd<0.68:
            mode = 'full_test_mode'
        else:
            mode = 'one_frame_mode'
        #mode = 'one_frame_mode'
        print(mode)
        #mode = 'hand_mode'
        agent = ResultGPT(mode)
        
        done = False
        score = 0
        if doom:
            env.new_episode()
            state_raw = env.get_state()
        state = prepare_features(state_raw)

        kills = 0
        items = 0
        health = 100
        hits = 0
        x_start = env.get_game_variable(GameVariable.POSITION_X)
        y_start = env.get_game_variable(GameVariable.POSITION_Y)
        z_start = env.get_game_variable(GameVariable.POSITION_Z)
        
        action = None
        t = -1
        while not done:
            # get action for the current state and go one step in environment
            t += 1
            action, gpt_memory_report = gpt_strategy_inertial(state_raw.screen_buffer,action, t)
            
            if doom:
                reward = env.make_action(actions[action])
                reward = env.get_game_variable(GameVariable.KILLCOUNT)-kills#я считаю как хочу
                reward += (env.get_game_variable(GameVariable.ITEMCOUNT)-items)*0.1
                reward += (env.get_game_variable(GameVariable.HITCOUNT)-hits)*0.15
                reward += (env.get_game_variable(GameVariable.HEALTH)-health)*0.005
                kills = env.get_game_variable(GameVariable.KILLCOUNT)
                items = env.get_game_variable(GameVariable.ITEMCOUNT)
                health = env.get_game_variable(GameVariable.HEALTH)
                hits = env.get_game_variable(GameVariable.HITCOUNT)
                if t%ACTION_PERIOD==0:
                    x = env.get_game_variable(GameVariable.POSITION_X)
                    y = env.get_game_variable(GameVariable.POSITION_Y)
                    z = env.get_game_variable(GameVariable.POSITION_Z)
                    rec = {'ep':ep, 't':int(t/ACTION_PERIOD),'img': state_raw.screen_buffer, 'kills':kills, 'items':items, 'health':health, 'hits':hits, 'action':num_to_action_desc[action], 'x':x - x_start, 'y':y - y_start, 'z':z - z_start}
                    for key in gpt_memory_report.keys():
                        rec[key] = gpt_memory_report[key]
                    logs_episodes.append(rec)
                if reward!=0:
                    print('reward',reward,flush=True)
                try:
                    next_state_raw = env.get_state()
                except Exception:
                    print('cannot parse frame')
                #next_state = np.reshape(next_state, [state_size[0], state_size[1], state_size[2]])
                done = env.is_episode_finished() or env.is_player_dead()
                #if env.is_player_dead():
                #    reward = -100
            else:
                next_state, reward, done, info = env.step(action)

            # if an action make the episode end, then gives penalty of -100

            if np.random.rand()<0.003:
                #запись автоаннотированной картинки
                encoded, obj_list, obj_matrix, description = prepare_features(state_raw,extended=True)
                img_orig = state_raw.screen_buffer
                b,g,r = img_orig[:,:,0], img_orig[:,:,1], img_orig[:,:,2]
                img_orig = np.dstack([r, g, b])
                
                img_name = f"autoannotated_{str(pd.Timestamp.now()).replace(' ', '_').replace(':', '_').replace('.', '_')}.png"
                description = description + '<END>\n'
                path_txt = '../gpt/chatgpt/data/imgs_descs.txt'
                path_img = f'../gpt/chatgpt/data/imgs/{img_name}'
                cv2.imwrite(path_img, img_orig)
                
                
                sting_add = f'<<{img_name}>> description:{description}'
                with codecs.open(path_txt, 'a', 'utf8') as f:
                    f.write(sting_add)
                #pass

            for obj in state_raw.labels:
                items_set.append(obj.object_name)
            if np.random.rand()<0.000:
                if not (state_raw is None):
                    #plt.imshow(state_raw.labels_buffer)
                    #plt.show()
                    encoded, obj_list, obj_matrix, description = prepare_features(state_raw,extended=True)
                    plt.imshow(state_raw.screen_buffer)
                    plt.show()
                    print(prepare_features(state_raw,True))

                    print('enemies')
                    plt.imshow(obj_matrix[:,:,0])
                    plt.show()
                    print('items')
                    plt.imshow(obj_matrix[:,:,1])
                    plt.show()
                    print('projectiles')
                    plt.imshow(obj_matrix[:,:,2])
                    plt.show()
                    print('corpses')
                    plt.imshow(obj_matrix[:,:,3])
                    plt.show()
                    print('other')
                    plt.imshow(obj_matrix[:,:,4])
                    plt.show()
                    print('depth')
                    plt.imshow(obj_matrix[:,:,5]/100)
                    plt.show()

            score += reward
            state_raw = next_state_raw

            if doom:
                if env.is_player_dead():
                    # Use this to respawn immediately after death, new state will be available.
                    env.respawn_player()

            if done:
                # every episode, plot the play time

                print("episode:", ep, "  score:", score, 'KILLS',kills,'HITS',hits,'ITEMS',items,flush=True)
                #if score>1:
                #    stop_train = True
         
        
    #записать реворды
    if 1:
        logs_episodes = pd.DataFrame(logs_episodes)
        logs_episodes['killpoints'] = logs_episodes['kills'] + logs_episodes['hits']*0.3
        logs_episodes['distance'] = logs_episodes['x']**2 + logs_episodes['y']**2 + logs_episodes['z']**2

        for start in range(logs_episodes.shape[0]):
            subtraj = logs_episodes.iloc[start:start + subtraj_len]
            ep = subtraj.ep.iloc[0]
            subtraj = subtraj.loc[subtraj.ep == ep]
            if subtraj.shape[0]<4:
                continue
            reward_d = {}
            reward_d['dhp'] = 0
            reward_d['distance'] = 0
            reward_d['killpoints'] = 0
            reward_d['items'] = 0
            for subtraj_len_cur in [subtraj_len, subtraj_len//2, subtraj_len//4]:
                subtraj_cur = subtraj.iloc[:subtraj_len_cur]
                reward_d['dhp'] += (subtraj_cur.health.iloc[-1] - subtraj_cur.health.iloc[0]) / subtraj_len_cur
                reward_d['distance'] += np.abs(subtraj_cur.distance.iloc[-1] - subtraj_cur.distance.iloc[0]) / subtraj_len_cur
                reward_d['killpoints'] += (subtraj_cur.killpoints.iloc[-1] - subtraj_cur.killpoints.iloc[0])/ subtraj_len_cur
                reward_d['items'] += (subtraj_cur['items'].iloc[-1] - subtraj_cur['items'].iloc[0]) / subtraj_len_cur
            reward_d['dhp'] /= (3 * 5)
            reward_d['distance'] /= (3 * 500000)
            reward_d['killpoints']/= (3 * 0.1)
            reward_d['items'] /= 3

            summary_simple = subtraj['summary_simple'].iloc[0]
            summary = subtraj['summary'].iloc[0]
            plan = ''
            len_frc_cur = np.min([subtraj.shape[0], actions_count])
            for t in range(len_frc_cur):
                if len(subtraj) > t:
                    plan += subtraj.action.iloc[t] + ', '

            #for goal in ['dhp', 'killpoints', 'items', 'distance']:
            for goal in ['killpoints']:
                r = reward_d[goal]
                path_txt = path_txt_memory
                #<<image>> summarize: (обычное описание)
                if summary_simple!='':
                    sting_add = f"{subtraj['text_for_summary_simple'].iloc[0]}summarize: {summary_simple}"
                    sting_add = sting_add.replace('description:<OUT>summarize:', 'summarize:<OUT>')
                    sting_add += str(reward_make_substr(r))
                    with codecs.open(path_txt, 'a', 'utf8') as f:
                        f.write(sting_add + '<END>\n')
                #(summary) <<image>>goal plan: (обычный план)
                sting_add = f"{subtraj['text_for_plan_complex'].iloc[0]}{plan}"
                sting_add = sting_add.replace('description:<OUT>summarize:', 'summarize:<OUT>')
                sting_add += str(reward_make_substr(r))

                with codecs.open(path_txt, 'a', 'utf8') as f:
                    f.write(sting_add + '<END>\n')
                #(summary) <<image>> summarize: (прошлое summary); (обычное описание)
                sting_add = f"{subtraj['text_for_summary_complex'].iloc[0]}summarize: {subtraj['summary'].iloc[0]}"
                sting_add = sting_add.replace('description:<OUT>summarize:', 'summarize:<OUT>')
                sting_add += str(reward_make_substr(r))

                with codecs.open(path_txt, 'a', 'utf8') as f:
                    f.write(sting_add + '<END>\n')

        print('written')

                        
        #отобрать рандомные траектории для прогноза
        logs_episodes = pd.DataFrame(logs_episodes)
        logs_episodes['killpoints'] = logs_episodes['kills'] + logs_episodes['hits']*0.3
        logs_episodes['distance'] = logs_episodes['x']**2 + logs_episodes['y']**2 + logs_episodes['z']**2
        #отобрать рандомные траектории для прогноза
        frc_count = 160
        for frc_num in range(frc_count):
            try:
                ep = int(np.random.rand()*logs_episodes.ep.max())
                traj_num = ep
                len_frc_cur = int(frc_len*np.random.rand()) + 1
                subtraj_len_cur = len_frc_cur
                t_start = int(np.random.rand() * (logs_episodes[logs_episodes.ep == traj_num].t.max() - subtraj_len_cur))
                img_start = logs_episodes[(logs_episodes.t==t_start)&(logs_episodes.ep==ep)].img.iloc[0]
                b,g,r = img_start[:,:,0], img_start[:,:,1], img_start[:,:,2]
                img_start = np.dstack([r, g, b])


                img_end = logs_episodes[(logs_episodes.t==t_start + len_frc_cur)&(logs_episodes.ep==ep)].img.iloc[0]
                b,g,r = img_end[:,:,0], img_end[:,:,1], img_end[:,:,2]
                img_end = np.dstack([r, g, b])

                #Прогноз картинки
                img_name1 = f"autofrcstart_{str(pd.Timestamp.now()).replace(' ', '_').replace(':', '_').replace('.', '_')}.png"
                path_img = f'../gpt/chatgpt/data/imgs/{img_name1}'
                cv2.imwrite(path_img, img_start)
                img_name2 = f"autofrcend_{str(pd.Timestamp.now()).replace(' ', '_').replace(':', '_').replace('.', '_')}.png"
                path_img = f'../gpt/chatgpt/data/imgs/{img_name2}'
                cv2.imwrite(path_img, img_end)

                plan = ''
                for i in range(len_frc_cur):
                    plan += logs_episodes[(logs_episodes.t==t_start+i)&(logs_episodes.ep==ep)].action.iloc[0] + ', '

                #forecast
                sting_add = f'<<{img_name1}>> plan:{plan} forecast img: <<{img_name2}>><END>\n'

                path_txt = '../gpt/chatgpt/data/imgs_descs.txt'
                with codecs.open(path_txt, 'a', 'utf8') as f:
                    f.write(sting_add)

                #goal agent or calculate diff between frames
                sting_add = f'<<{img_name1}>> task: <<{img_name2}>> plan:{plan}<END>\n'

                path_txt = '../gpt/chatgpt/data/imgs_descs.txt'
                with codecs.open(path_txt, 'a', 'utf8') as f:
                    f.write(sting_add)

                #Прогноз переменных
                log_start = logs_episodes[(logs_episodes.t==t_start)&(logs_episodes.ep==ep)]
                log_end = logs_episodes[(logs_episodes.t==t_start + len_frc_cur)&(logs_episodes.ep==ep)]
                kills = int(log_end['kills'].iloc[0] - log_start['kills'].iloc[0])
                hits = int(log_end['hits'].iloc[0] - log_start['hits'].iloc[0])
                dhp = int(log_end['health'].iloc[0] - log_start['health'].iloc[0])
                items = int(log_end['items'].iloc[0] - log_start['items'].iloc[0])
                sting_add = f'<<{img_name1}>> plan:{plan} forecast vars: kills:{kills}, hits:{hits}, delta hp:{dhp}, items:{items}<END>\n'
                with codecs.open(path_txt, 'a', 'utf8') as f:
                    f.write(sting_add)
            except Exception as e:
                print('frc error,', e)
        print('written frc')

    env.close()

Initializing doom...
Doom initialized.
one_frame_mode
plan in DATASET: <657><541><945><642><788><729><608><468><140><672><681><675><217><498><562><7> description:<OUT><END>

<IN><772><313><577><761><500><250><363><925><425
plan in DATASET: 985><494><922><129><552><362><469><608><39><945><230><576><316><217><498><562><414> description:<OUT><OUT><END>

<IN><250><821><556><695><965><618><72>
reward 0.15
reward -0.15
reward 2.05
reward -0.015
reward -0.03
reward -0.075
reward 1.45
reward -0.06
reward -0.03
reward -0.105
reward -0.09
episode: 0   score: 3.095 KILLS 2.0 HITS 11.0 ITEMS 0.0
full_test_mode


In [ ]:
text = '<IN><325><730><771><710><916><992><710><26><556><168><517><498><414><845><92><414><663><534><168><323><988><695><526><589><72><287><884><965><868><589><803><897><12><695><263><7><389><462><107><358><187><828><200><202><328><870><746><40><269><913><148><269><417><698><358><245><263><518><685><220><432><66><243><657><58><644><892><129><608><382><624><945><468><140><230><382><371><608><784><713><7> kill all plan:<OUT>'
index = lines.find(text)
#print('index', index)
offset = 250
l_offset = 350
if index>=0:
    print('plan in DATASET:', lines[index + 81*4 - 1 - l_offset: index + 81*4 + offset])

In [ ]:
#subtraj_len = 140
#subtraj_selection = 60

In [ ]:
logs_episodes = pd.DataFrame(logs_episodes)
logs_episodes.iloc[:2]

In [ ]:
logs_episodes.iloc[-10:]['summary_simple'].values

In [ ]:
logs_episodes.iloc[:10]['text_for_summary_simple'].values

In [ ]:
logs_episodes.iloc[-15:]['summary_simple'].values

In [ ]:
logs_episodes.iloc[-15:]['text_for_plan_complex'].values

In [ ]:
goal_description = {'dhp':'save hp', 'killpoints':'kill all', 'items':'pickup items', 'distance': 'scouting'}

In [ ]:
recoms_count_dense = 1000

In [ ]:
#path_txt_memory = '../gpt/chatgpt/data/imgs_descs_memory_20.txt'

In [ ]:
#записать реворды
if 1:
    logs_episodes = pd.DataFrame(logs_episodes)
    logs_episodes['killpoints'] = logs_episodes['kills'] + logs_episodes['hits']*0.3
    logs_episodes['distance'] = logs_episodes['x']**2 + logs_episodes['y']**2 + logs_episodes['z']**2

    for start in range(logs_episodes.shape[0]):
        subtraj = logs_episodes.iloc[start:start + subtraj_len]
        ep = subtraj.ep.iloc[0]
        subtraj = subtraj.loc[subtraj.ep == ep]
        if subtraj.shape[0]<4:
            continue
        reward_d = {}
        reward_d['dhp'] = 0
        reward_d['distance'] = 0
        reward_d['killpoints'] = 0
        reward_d['items'] = 0
        for subtraj_len_cur in [subtraj_len, subtraj_len//2, subtraj_len//4]:
            subtraj_cur = subtraj.iloc[:subtraj_len_cur]
            reward_d['dhp'] += (subtraj_cur.health.iloc[-1] - subtraj_cur.health.iloc[0]) / subtraj_len_cur
            reward_d['distance'] += np.abs(subtraj_cur.distance.iloc[-1] - subtraj_cur.distance.iloc[0]) / subtraj_len_cur
            reward_d['killpoints'] += (subtraj_cur.killpoints.iloc[-1] - subtraj_cur.killpoints.iloc[0])/ subtraj_len_cur
            reward_d['items'] += (subtraj_cur['items'].iloc[-1] - subtraj_cur['items'].iloc[0]) / subtraj_len_cur
        reward_d['dhp'] /= (3 * 5)
        reward_d['distance'] /= (3 * 500000)
        reward_d['killpoints']/= (3 * 0.1)
        reward_d['items'] /= 3

        summary_simple = subtraj['summary_simple'].iloc[0]
        summary = subtraj['summary'].iloc[0]
        plan = ''
        len_frc_cur = np.min([subtraj.shape[0], actions_count])
        for t in range(len_frc_cur):
            if len(subtraj) > t:
                plan += subtraj.action.iloc[t] + ', '

        #for goal in ['dhp', 'killpoints', 'items', 'distance']:
        for goal in ['killpoints']:
            r = reward_d[goal]
            path_txt = path_txt_memory
            #<<image>> summarize: (обычное описание)
            if summary_simple!='':
                sting_add = f"{subtraj['text_for_summary_simple'].iloc[0]}summarize: {summary_simple}"
                sting_add = sting_add.replace('description:<OUT>summarize:', 'summarize:<OUT>')
                sting_add += str(reward_make_substr(r))
                with codecs.open(path_txt, 'a', 'utf8') as f:
                    f.write(sting_add + '<END>\n')
            #(summary) <<image>>goal plan: (обычный план)
            sting_add = f"{subtraj['text_for_plan_complex'].iloc[0]}{plan}"
            sting_add = sting_add.replace('description:<OUT>summarize:', 'summarize:<OUT>')
            sting_add += str(reward_make_substr(r))

            with codecs.open(path_txt, 'a', 'utf8') as f:
                f.write(sting_add + '<END>\n')
            #(summary) <<image>> summarize: (прошлое summary); (обычное описание)
            sting_add = f"{subtraj['text_for_summary_complex'].iloc[0]}summarize: {subtraj['summary'].iloc[0]}"
            sting_add = sting_add.replace('description:<OUT>summarize:', 'summarize:<OUT>')
            sting_add += str(reward_make_substr(r))
            if '-' in reward_make_substr(r):
                1/0
            with codecs.open(path_txt, 'a', 'utf8') as f:
                f.write(sting_add + '<END>\n')
            print('r', r)

    print('written')

In [ ]:
#отобрать рандомные траектории для прогноза
logs_episodes = pd.DataFrame(logs_episodes)
logs_episodes['killpoints'] = logs_episodes['kills'] + logs_episodes['hits']*0.3
logs_episodes['distance'] = logs_episodes['x']**2 + logs_episodes['y']**2 + logs_episodes['z']**2
#отобрать рандомные траектории для прогноза
frc_count = 40
for frc_num in range(frc_count):
    ep = int(np.random.rand()*logs_episodes.ep.max())
    traj_num = ep
    len_frc_cur = int(frc_len*np.random.rand()) + 1
    subtraj_len_cur = len_frc_cur
    t_start = int(np.random.rand() * (logs_episodes[logs_episodes.ep == traj_num].t.max() - subtraj_len_cur))
    img_start = logs_episodes[(logs_episodes.t==t_start)&(logs_episodes.ep==ep)].img.iloc[0]
    b,g,r = img_start[:,:,0], img_start[:,:,1], img_start[:,:,2]
    img_start = np.dstack([r, g, b])


    img_end = logs_episodes[(logs_episodes.t==t_start + len_frc_cur)&(logs_episodes.ep==ep)].img.iloc[0]
    b,g,r = img_end[:,:,0], img_end[:,:,1], img_end[:,:,2]
    img_end = np.dstack([r, g, b])

    #Прогноз картинки
    img_name1 = f"autofrcstart_{str(pd.Timestamp.now()).replace(' ', '_').replace(':', '_').replace('.', '_')}.png"
    path_img = f'../gpt/chatgpt/data/imgs/{img_name1}'
    cv2.imwrite(path_img, img_start)
    img_name2 = f"autofrcend_{str(pd.Timestamp.now()).replace(' ', '_').replace(':', '_').replace('.', '_')}.png"
    path_img = f'../gpt/chatgpt/data/imgs/{img_name2}'
    cv2.imwrite(path_img, img_end)

    plan = ''
    for i in range(len_frc_cur):
        plan += logs_episodes[(logs_episodes.t==t_start+i)&(logs_episodes.ep==ep)].action.iloc[0] + ', '

    #forecast
    sting_add = f'<<{img_name1}>> plan:{plan} forecast img: <<{img_name2}>><END>\n'

    path_txt = '../gpt/chatgpt/data/imgs_descs.txt'
    with codecs.open(path_txt, 'a', 'utf8') as f:
        f.write(sting_add)

    #goal agent or calculate diff between frames
    sting_add = f'<<{img_name1}>> task: <<{img_name2}>> plan:{plan}<END>\n'

    path_txt = '../gpt/chatgpt/data/imgs_descs.txt'
    with codecs.open(path_txt, 'a', 'utf8') as f:
        f.write(sting_add)

    #Прогноз переменных
    log_start = logs_episodes[(logs_episodes.t==t_start)&(logs_episodes.ep==ep)]
    log_end = logs_episodes[(logs_episodes.t==t_start + len_frc_cur)&(logs_episodes.ep==ep)]
    kills = int(log_end['kills'].iloc[0] - log_start['kills'].iloc[0])
    hits = int(log_end['hits'].iloc[0] - log_start['hits'].iloc[0])
    dhp = int(log_end['health'].iloc[0] - log_start['health'].iloc[0])
    items = int(log_end['items'].iloc[0] - log_start['items'].iloc[0])
    sting_add = f'<<{img_name1}>> plan:{plan} forecast vars: kills:{kills}, hits:{hits}, delta hp:{dhp}, items:{items}<END>\n'
    with codecs.open(path_txt, 'a', 'utf8') as f:
        f.write(sting_add)
    print('written frc')

In [ ]:
1/0

In [ ]:
#запись в файл

In [ ]:
logs_episodes = pd.DataFrame(logs_episodes)
idx = logs_episodes.ep==0
video = logs_episodes['img'].values
video

In [ ]:
#video = video[-153:]

In [ ]:
#записать видео
#draw video
import cv2
name = 'map13'
path = f"./out_videos/{name}.mp4"
writer = cv2.VideoWriter(path,cv2.VideoWriter_fourcc(*"MP4V"),30//8,(512,512))
for i in range(0,len(video),1):
    resize = cv2.resize(np.array(video[i]), (512, 512))
    frame = resize*0
    frame[:,:,0],frame[:,:,1],frame[:,:,2] = resize[:,:,2],resize[:,:,1], resize[:,:,0]
    writer.write(frame)
writer.release()

In [ ]:
import os
from moviepy.editor import ImageSequenceClip

video = np.array(video)
path = f"./out_videos/{name}.gif"
def gif(filename, array, fps=10, scale=1.0):
    """Creates a gif given a stack of images using moviepy
    Notes
    -----
    works with current Github version of moviepy (not the pip version)
    https://github.com/Zulko/moviepy/commit/d4c9c37bc88261d8ed8b5d9b7c317d13b2cdf62e
    Usage
    -----
    >>> X = randn(100, 64, 64)
    >>> gif('test.gif', X)
    Parameters
    ----------
    filename : string
        The filename of the gif to write to
    array : array_like
        A numpy array that contains a sequence of images
    fps : int
        frames per second (default: 10)
    scale : float
        how much to rescale each image by (default: 1.0)
    """

    # ensure that the file has the .gif extension
    fname, _ = os.path.splitext(filename)
    filename = fname + '.gif'

    # copy into the color dimension if the images are black and white
    if array.ndim == 3:
        array = array[..., np.newaxis] * np.ones(3)

    # make the moviepy clip
    clip = ImageSequenceClip(list(array), fps=fps).resize(scale)
    clip.write_gif(filename, fps=fps)
    return clip

gif(path, video, fps=10, scale=1.0)

In [ ]:
Выяснить, сколько времени идёт preprocess_vqgan на GPU и на CPU. GPU: 0 days 00:00:00.016816540 CPU: 0 days 00:00:00.268827660
Выяснить, сколько времени идёт answer на GPU и на CPU. GPU: 0 days 00:00:01.682268666 CPU: 0 days 00:00:08.292097500
Выяснить, сколько времени идёт encode, generate, decode на GPU
encode 0 days 00:00:00.000166833
model.generate 0 days 00:00:01.663960
decode 0 days 00:00:00.000083500
model.generate 
tokp=1 gener_len=1 0 days 00:00:00.041204083
tokp=1 max_length=1 0 days 00:00:00.043289416
tokp=10 gener_len=1 0 days 00:00:00.041537666
tokp=10 max_length=1 0 days 00:00:00.042955833
tokp=1 gener_len=10 0 days 00:00:00.273122166
tokp=1 max_length=10 0 days 00:00:00.042081250
tokp=10 gener_len=100 0 days 00:00:02.356350416 !!!
tokp=10 max_length=100 0 days 00:00:00.379942250
Время генерации ~gener_len и ~tokp
Итого, генерация занимает 1.6 для генерации описания (70 токенов. Если сделать 40 токенов, то 1.08). Если генерировать только план, получится 00.27